In [1]:
import os
import numpy as np
import pandas as pd
import torch
import torch.nn as nn

In [2]:
class fixed_gru(nn.Module):
    def __init__(self, num_steps, emb_dim):
        super().__init__()
        # update gate
        self.i2h_update = nn.Linear(emb_dim, emb_dim)
        self.h2h_update = nn.Linear(emb_dim, emb_dim)

        # reset gate
        self.i2h_reset = nn.Linear(emb_dim, emb_dim)
        self.h2h_reset = nn.Linear(emb_dim, emb_dim)

        # candidate hidden state
        self.i2h = nn.Linear(emb_dim, emb_dim)
        self.h2h = nn.Linear(emb_dim, emb_dim)

        self.num_steps = num_steps


    def forward(self, txt):
        res = []
        res_intermediate = []
        for i in range(self.num_steps):
            if i == 0:
                output = torch.tanh(self.i2h(txt[:, i])).unsqueeze(1)
            else:
                # compute update and reset gates
                update = torch.sigmoid(self.i2h_update(txt[:, i]) + self.h2h_update(res[i-1]))
                reset = torch.sigmoid(self.i2h_reset(txt[:, i]) + self.h2h_reset(res[i-1]))

                # compute candidate hidden state
                gated_hidden = reset * res[i-1]
                p1 = self.i2h(txt[:, i])
                p2 = self.h2h(gated_hidden)
                hidden_cand = torch.tanh(p1 + p2)

                # use gates to interpolate hidden state
                zh = update * hidden_cand
                zhm1 = ((update * -1) + 1) * res[i-1]
                output = zh + zhm1

            res.append(output)
            res_intermediate.append(output)

        res = torch.cat(res, dim=1)
        res = torch.mean(res, dim=1)
        return res

In [3]:
class char_cnn_rnn(nn.Module):
    def __init__(self):
        super().__init__()
        rnn=fixed_gru
        use_maxpool3=True
        self.use_maxpool3=use_maxpool3
        rnn_num_steps=8
        rnn_dim=512

        # network setup
        # (B, 70, 201)
        self.conv1 = nn.Conv1d(70, 384, kernel_size=4)
        self.threshold1 = nn.Threshold(1e-6, 0)
        self.maxpool1 = nn.MaxPool1d(kernel_size=3, stride=3)
        # (B, 384, 66)
        self.conv2 = nn.Conv1d(384, 512, kernel_size=4)
        self.threshold2 = nn.Threshold(1e-6, 0)
        self.maxpool2 = nn.MaxPool1d(kernel_size=3, stride=3)
        # (B, 512, 21)
        self.conv3 = nn.Conv1d(512, rnn_dim, kernel_size=4)
        self.threshold3 = nn.Threshold(1e-6, 0)
        if use_maxpool3:
            self.maxpool3 = nn.MaxPool1d(kernel_size=3,stride=2)
        # (B, rnn_dim, rnn_num_steps)
        self.rnn = rnn(num_steps=rnn_num_steps, emb_dim=rnn_dim)
        # (B, rnn_dim)
        self.emb_proj = nn.Linear(rnn_dim, 1024)
        # (B, 1024)


    def forward(self, txt):
        # temporal convolutions
#         print(txt.shape)
        out = self.conv1(txt)
        out = self.threshold1(out)
        out = self.maxpool1(out)

        out = self.conv2(out)
        out = self.threshold2(out)
        out = self.maxpool2(out)

        out = self.conv3(out)
        out = self.threshold3(out)
        if self.use_maxpool3:
            out = self.maxpool3(out)

        # recurrent computation
        out = out.permute(0, 2, 1)
        out = self.rnn(out)

        # linear projection
        out = self.emb_proj(out)

        return out

In [73]:
def prepare_text(string, max_str_len=201):
    '''
    Converts a text description from string format to one-hot tensor format.
    '''
    labels = str_to_labelvec(string, max_str_len)
    one_hot = labelvec_to_onehot(labels)
    return one_hot



def str_to_labelvec(string, max_str_len):
    string = string.lower()
    alphabet = "abcdefghijklmnopqrstuvwxyz0123456789-,;.!?:'\"/\\|_@#$%^&*~`+-=<>()[]{} "
    alpha_to_num = {k:v+1 for k,v in zip(alphabet, range(len(alphabet)))}
    labels = torch.zeros(max_str_len).long()
    max_i = min(max_str_len, len(string))
    for i in range(max_i):
        labels[i] = alpha_to_num.get(string[i], alpha_to_num[' '])

    return labels



def labelvec_to_onehot(labels):
    labels2 = torch.LongTensor(labels).unsqueeze(0)
#     print(labels2.shape)
    one_hot = torch.zeros(labels2.size(1), 71).scatter_(1, labels2, 1.)
    # ignore zeros in one-hot mask (position 0 = empty one-hot)
    one_hot = one_hot[:, 1:]
    one_hot = one_hot.permute(1,0)
#     print(one_hot.shape)
    return one_hot



def onehot_to_labelvec(tensor):
    labels = torch.zeros(tensor.size(1), dtype=torch.long)
    val, idx = torch.nonzero(tensor).split(1, dim=1)
    labels[idx] = val+1
    return labels



def labelvec_to_str(labels):
    '''
    Converts a text description from one-hot tensor format to string format.
    '''
    alphabet = "abcdefghijklmnopqrstuvwxyz0123456789-,;.!?:'\"/\\|_@#$%^&*~`+-=<>()[]{} "
    string = [alphabet[x-1] for x in labels if x > 0]
    string = ''.join(string)
    return string

In [61]:
import random

In [6]:
def rng_init(seed):
    random.seed(seed)
    #np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True



def init_weights(m):
    if isinstance(m, torch.nn.Linear):
        torch.nn.init.uniform_(m.weight, a=-0.08, b=0.08)
        torch.nn.init.uniform_(m.bias, a=-0.08, b=0.08)
    elif isinstance(m, torch.nn.Conv1d):
        torch.nn.init.uniform_(m.weight, a=-0.08, b=0.08)
        torch.nn.init.uniform_(m.bias, a=-0.08, b=0.08)

In [7]:
import argparse
from collections import OrderedDict

from tqdm import tqdm
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
from datetime import datetime
from torch.utils.tensorboard import SummaryWriter

2024-03-04 06:45:19.578013: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-04 06:45:19.578114: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-04 06:45:19.711343: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [38]:
class MultimodalDataset(Dataset):
    '''
    Preprocessed Caltech-UCSD Birds 200-2011 and Oxford 102 Category Flowers
    datasets, used in ``Learning Deep Representations of Fine-grained Visual
    Descriptions``.

    Download data from: https://github.com/reedscot/cvpr2016.

    Arguments:
        data_dir (string): path to directory containing dataset files.
        split (string): which data split to load.
    '''
    def __init__(self, data_dir):
        super().__init__()
        self.split_classes = ['dresses','jackets and coats','jeans','pants','shirts','shorts','skirts','suits and blazers','sweaters','tops']
        self.nclass = len(self.split_classes)

        self.data = {}
        self.num_instances = 0
        for cls in self.split_classes:
#             print(cls)
            path_imgs=f'{data_dir}/img-emb/img-emb/train/{cls}.t7'
            path_txts=f'{data_dir}/text-to-label/text-to-label/train/{cls}.t7'
            cls_imgs = torch.Tensor(torch.load(path_imgs))
            cls_txts = torch.LongTensor(torch.load(path_txts))
#             print(cls_imgs.size,cls_txts.size)
            self.data[cls] = (cls_imgs, cls_txts)

            self.num_instances += cls_imgs.size(0)


    def __len__(self):
        # WARNING: this number is somewhat arbitrary, since we do not
        # necessarily use all instances in an epoch
        return self.num_instances


    def __getitem__(self, index):
        cls_id = torch.randint(self.nclass, (1,))
        cls = self.split_classes[cls_id]
        cls_imgs, cls_txts = self.data[cls]

#         id_txt = torch.randint(cls_txts.size(2), (1,))
#         id_instance = torch.randint(cls_txts.size(0), (1,))
#         id_view = torch.randint(cls_imgs.size(2), (1,))

#         img = cls_imgs[id_instance, :, id_view].squeeze()
#         txt = cls_txts[id_instance, :, id_txt].squeeze()
        
        id=torch.randint(cls_txts.size(0),(1,))
#         id_view=torch.randint(cls_imgs.size(0),(1,))
        img = cls_imgs[id]
        txt = cls_txts[id]
        
        txt = labelvec_to_onehot(txt)

        return {'img': img, 'txt': txt}

In [39]:
def sje_loss(feat1, feat2):
    ''' Structured Joint Embedding Loss '''
    # similarity score matrix (rows: fixed feat2, columns: fixed feat1)
    scores = torch.matmul(feat2, feat1.t()) # (B, B)
    # diagonal: matching pairs
    diagonal = scores.diag().view(scores.size(0), 1) # (B, 1)
    # repeat diagonal scores on rows
    diagonal = diagonal.expand_as(scores) # (B, B)
    # calculate costs
    cost = (1 + scores - diagonal).clamp(min=0) # (B, B)
    # clear diagonals (matching pairs are not used in loss computation)
    cost[torch.eye(cost.size(0)).bool()] = 0 # (B, B) for torch==1.2.0
#     cost[torch.eye(cost.size(0), dtype=torch.uint8)] = 0 # (B, B)
    # sum and average costs
    denom = cost.size(0) * cost.size(1)
    loss = cost.sum() / denom

    # batch accuracy
    max_ids = torch.argmax(scores, dim=1)
    ground_truths = torch.LongTensor(range(scores.size(0))).to(feat1.device)
    num_correct = (max_ids == ground_truths).sum().float()
    accuracy = 100 * num_correct / cost.size(0)

    return loss, accuracy

In [40]:
def main():
    rng_init(42)
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    data_dir='/kaggle/input/embeddings/'
    dataset = MultimodalDataset(data_dir)
    loader = DataLoader(dataset, batch_size=50, shuffle=True,
            num_workers=1, pin_memory=True)
    loader_len = len(loader)
#     print("Done dataset")

    os.makedirs('/kaggle/working/checkpoints', exist_ok=True)
    timestamp = datetime.now().strftime('%Y_%m_%d_%H_%M_%S')
    model_name = '{}_{:.5f}_{}_{}_{}.pth'.format('checkpoint',
            0.0004, True, 'train', timestamp)
    ckpt_path = os.path.join('/kaggle/working/checkpoints', model_name)
    writer = SummaryWriter('/kaggle/working/checkpoints')

    net_txt = char_cnn_rnn().to(device)
    first_time=True
    net_txt.apply(init_weights)

    optim_txt = torch.optim.RMSprop(net_txt.parameters(), lr=0.0004)
    sched_txt = torch.optim.lr_scheduler.ExponentialLR(optim_txt,0.98)

    acc1_smooth = acc2_smooth = 0

    for epoch in tqdm(range(300), position=1):
        for i, data in enumerate(tqdm(loader, position=0)):
            iter_num = (epoch * loader_len) + i + 1

            net_txt.train()
            img = data['img'].squeeze().to(device)
            txt = data['txt'].to(device)
            feat_txt = net_txt(txt)
            feat_img = img
#             print(feat_img.shape)

            loss1, acc1 = sje_loss(feat_txt, feat_img)
            loss2 = acc2 = 0
            loss2, acc2 = sje_loss(feat_img, feat_txt)
            loss = loss1 + loss2

            acc1_smooth = 0.99 * acc1_smooth + 0.01 * acc1
            acc2_smooth = 0.99 * acc2_smooth + 0.01 * acc2

            net_txt.zero_grad()
            loss.backward()
            optim_txt.step()

            writer.add_scalar('train/loss1', loss1.item(), iter_num)
            writer.add_scalar('train/loss2', loss2.item(), iter_num)
            writer.add_scalar('train/acc1', acc1, iter_num)
            writer.add_scalar('train/acc2', acc2, iter_num)
            writer.add_scalar('train/acc1_smooth', acc1_smooth, iter_num)
            writer.add_scalar('train/acc2_smooth', acc2_smooth, iter_num)
            writer.add_scalar('train/lr', sched_txt.get_lr()[0], iter_num)

            if (iter_num % 100) == 0:
                run_info = (
                        'epoch: [{:3d}/{:3d}] | step: [{:4d}/{:4d}] | '
                        'loss: {:.4f} | loss1: {:.4f} | loss2: {:.4f} | '
                        'acc1: {:.2f} | acc2: {:.2f} | '
                        'acc1_smooth: {:.3f} | acc2_smooth: {:.3f} | '
                        'lr: {:.8f}'
                ).format(epoch+1, 300, (i+1), loader_len,
                        loss, loss1, loss2,
                        acc1, acc2,
                        acc1_smooth, acc2_smooth,
                        sched_txt.get_lr()[0])
                tqdm.write(run_info)

        net_txt.eval()
        tqdm.write('Saving checkpoint to: {}'.format(ckpt_path))
        torch.save(net_txt.state_dict(), ckpt_path)

        sched_txt.step()

    writer.close()

In [ ]:
main()

In [43]:
%cd /kaggle/working/checkpoints
!ls

/kaggle/working/checkpoints
checkpoint_0.00040_True_train_2024_03_04_06_45_49.pth
checkpoint_0.00040_True_train_2024_03_04_09_37_24.pth
events.out.tfevents.1709534749.b76a8154565b.34.0
events.out.tfevents.1709545044.b76a8154565b.34.1


In [80]:
FileLink(r'checkpoint_0.00040_True_train_2024_03_04_09_37_24.pth')

/kaggle/working/checkpoints/checkpoint_0.00040_True_train_2024_03_04_09_37_24.pth

In [ ]:
%tensorboard --logdir /kaggle/working/checkpoints/checkpoint

In [95]:
"""
    Encoder for preprocessed Caltech-UCSD Birds 200-2011 and Oxford 102
    Category Flowers datasets, used in ``Learning Deep Representations of
    Fine-grained Visual Descriptions``.

    Warning: if you decide to not use all sentences (i.e., num_txts_eval > 0),
    sentences will be randomly sampled and their features will be averaged to
    provide a class representation. This means that the evaluation procedures
    should be performed multiple times (using different seeds) to account for
    this randomness.

    Arguments:
        net_txt (torch.nn.Module): text processing network.
        net_img (torch.nn.Module): image processing network.
        data_dir (string): path to directory containing dataset files.
        split (string): which data split to load.
        num_txts_eval (int): number of textual descriptions to use for each
            class (0 = use all). The embeddings are averaged per-class.
        batch_size (int): batch size to split data processing into chunks.
        device (torch.device): which device to do computation in.

    Returns:
        cls_feats_img (list of torch.Tensor): list containing precomputed image
            features for each image, separated by class.
        cls_feats_txt (torch.Tensor): tensor containing precomputed (and
            averaged) textual features for each class.
        cls_list (list of string): list of class names.
"""
def encode_data(net_txt, net_img, data_dir, split, num_txts_eval, batch_size, device):
    cls_list = ['dresses','jackets and coats','jeans','pants','shirts','shorts','skirts','suits and blazers','sweaters','tops']

    cls_feats_img = []
    cls_feats_txt = []
    for cls in cls_list:
        # prepare image data
        data_img_path = f'{data_dir}/img-emb/img-emb/val/{cls}.t7'
        data_img = torch.Tensor(torch.load(data_img_path))
        
        feats_img = data_img[:, :].to(device)
        if net_img is not None:
            with torch.no_grad():
                feats_img = net_img(feats_img)
        cls_feats_img.append(feats_img)

        # prepare text data
        data_txt_path = f'{data_dir}/text-to-label/text-to-label/train/{cls}.t7'
        data_txt = torch.LongTensor(torch.load(data_txt_path))

        # select T texts from all instances to represent this class
#         data_txt = data_txt.permute(0, 2, 1)
#         total_txts = data_txt.size(0) * data_txt.size(1)
#         data_txt = data_txt.contiguous().view(total_txts, -1)
        if num_txts_eval > 0:
            num_txts_eval = min(num_txts_eval, total_txts)
            id_txts = torch.randperm(data_txt.size(0))[:num_txts_eval]
            data_txt = data_txt[id_txts]

        # convert to one-hot tensor to run through network
        # TODO: adapt code to support batched version
        txt_onehot = []
        for txt in data_txt:
            txt_onehot.append(labelvec_to_onehot(txt))
        txt_onehot = torch.stack(txt_onehot)

        # if we use a lot of text descriptions, it will not fit in gpu memory
        # separate instances into mini-batches to process them using gpu
        feats_txt = []
        for batch in torch.split(txt_onehot, batch_size, dim=0):
            with torch.no_grad():
                out = net_txt(batch.to(device))
            feats_txt.append(out)

        # average the outputs
        feats_txt = torch.cat(feats_txt, dim=0).mean(dim=0)
        cls_feats_txt.append(feats_txt)

    cls_feats_txt = torch.stack(cls_feats_txt, dim=0)

    return cls_feats_img, cls_feats_txt, cls_list


In [96]:
  '''
    Classification evaluation.

    Arguments:
        cls_feats_img (list of torch.Tensor): list containing precomputed image
            features for each image, separated by class.
        cls_feats_txt (torch.Tensor): tensor containing precomputed (and
            averaged) textual features for each class.
        cls_list (list of string): list of class names.

    Returns:
        avg_acc (float): percentage of correct classifications for all classes.
        cls_stats (OrderedDict): dictionary whose keys are class names and each
            entry is a dictionary containing the 'total' of images for the
            class and the number of 'correct' classifications.
    '''
def eval_classify(cls_feats_img, cls_feats_txt, cls_list):
    cls_stats = OrderedDict()
    for i, cls in enumerate(cls_list):
        feats_img = cls_feats_img[i]
        scores = torch.matmul(feats_img, cls_feats_txt.t())
        max_ids = torch.argmax(scores, dim=1).to('cpu')
        ground_truths = torch.LongTensor(scores.size(0)).fill_(i)
        num_correct = (max_ids == ground_truths).sum().item()
        cls_stats[cls] = {'correct': num_correct, 'total': ground_truths.size(0)}

    total = sum([stats['total'] for _, stats in cls_stats.items()])
    total_correct = sum([stats['correct'] for _, stats in cls_stats.items()])
    avg_acc = total_correct / total
    return avg_acc, cls_stats

In [97]:
def eval_retrieval(cls_feats_img, cls_feats_txt, cls_list, k_values=[1,5,10,50]):
    '''
    Retrieval evaluation (Average Precision).

    Arguments:
        cls_feats_img (list of torch.Tensor): list containing precomputed image
            features for each image, separated by class.
        cls_feats_txt (torch.Tensor): tensor containing precomputed (and
            averaged) textual features for each class.
        cls_list (list of string): list of class names.
        k_values (list, optional): list of k-values to use for evaluation.

    Returns:
        map_at_k (OrderedDict): dictionary whose keys are the k_values and the
            values are the mean Average Precision (mAP) for all classes.
        cls_stats (OrderedDict): dictionary whose keys are class names and each
            entry is a dictionary whose keys are the k_values and the values
            are the Average Precision (AP) per class.
    '''
    total_num_cls = cls_feats_txt.size(0)
    total_num_img = sum([feats.size(0) for feats in cls_feats_img])
    scores = torch.zeros(total_num_cls, total_num_img)
    matches = torch.zeros(total_num_cls, total_num_img)

    for i, cls in enumerate(cls_list):
        start_id = 0
        for j, feats_img in enumerate(cls_feats_img):
            end_id = start_id + feats_img.size(0)
            scores[i, start_id:end_id] = torch.matmul(feats_img, cls_feats_txt[i])
            if i == j: matches[i, start_id:end_id] = 1
            start_id = start_id + feats_img.size(0)

    for i, s in enumerate(scores):
        _, inds = torch.sort(s, descending=True)
        matches[i] = matches[i, inds]

    map_at_k = OrderedDict()
    for k in k_values:
        map_at_k[k] = torch.mean(matches[:, 0:k]).item()

    cls_stats = OrderedDict()
    for i, cls in enumerate(cls_list):
        ap_at_k = OrderedDict()
        for k in k_values:
            ap_at_k[k] = torch.mean(matches[i, 0:k]).item()
        cls_stats[cls] = ap_at_k

    return map_at_k, cls_stats

In [81]:
import pickle

In [98]:
def evaluate():
    rng_init(42)
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    net_txt = char_cnn_rnn()
    model_path='/kaggle/working/checkpoints/checkpoint_0.00040_True_train_2024_03_04_09_37_24.pth'
    net_txt.load_state_dict(torch.load(model_path, map_location=device))
#     with open('/kaggle/working/utils/model.pickle','wb') as f1:
#         pickle.dump(net_txt,f1)
#     f1.close()
    
#     with open('/kaggle/working/utils/weights.pickle','wb') as f2:
#         pickle.dump(torch.load(model_path),f2)
#     f2.close()
    print('Loaded')
    net_txt = net_txt.to(device)
    net_txt.eval()
    
    data_dir='/kaggle/input/embeddings'
    cls_feats_img, cls_feats_txt, cls_list = encode_data(net_txt, None, data_dir,
            'train', 0, 40, device)

    mean_ap, cls_stats = eval_retrieval(cls_feats_img, cls_feats_txt, cls_list)
    print('----- RETRIEVAL -----')
        
    print('  PER CLASS:')
    for name, stats in cls_stats.items():
        print(name)
        for k, ap in stats.items():
            print('{:.4f}: AP@{}'.format(ap, k))
    print()

    print('  mAP:')
    for k, v in mean_ap.items():
        print('{:.4f}: mAP@{}'.format(v, k))
    print('---------------------')
    print()

    avg_acc, cls_stats = eval_classify(cls_feats_img, cls_feats_txt, cls_list)
    print('--- CLASSIFICATION --')
    
    print('  PER CLASS:')
    for name, stats in cls_stats.items():
        print('{:.4f}: {}'.format(stats['correct'] / stats['total'], name))
    print()

    print('Average top-1 accuracy: {:.4f}'.format(avg_acc))
    print('---------------------')
    
    
    print()
    with open('/kaggle/working/utils/char-CNN-RNN-embeddings-val.pickle','wb') as f3:
        pickle.dump(cls_feats_txt,f3)
    f3.close()

In [99]:
evaluate()

Loaded
----- RETRIEVAL -----
  PER CLASS:
dresses
0.0000: AP@1
0.0000: AP@5
0.1000: AP@10
0.1000: AP@50
jackets and coats
1.0000: AP@1
1.0000: AP@5
1.0000: AP@10
0.9200: AP@50
jeans
1.0000: AP@1
1.0000: AP@5
1.0000: AP@10
0.9600: AP@50
pants
1.0000: AP@1
1.0000: AP@5
1.0000: AP@10
1.0000: AP@50
shirts
1.0000: AP@1
1.0000: AP@5
1.0000: AP@10
0.8200: AP@50
shorts
1.0000: AP@1
0.6000: AP@5
0.4000: AP@10
0.3400: AP@50
skirts
0.0000: AP@1
0.4000: AP@5
0.5000: AP@10
0.4000: AP@50
suits and blazers
0.0000: AP@1
0.6000: AP@5
0.7000: AP@10
0.6200: AP@50
sweaters
1.0000: AP@1
1.0000: AP@5
1.0000: AP@10
0.9000: AP@50
tops
1.0000: AP@1
1.0000: AP@5
1.0000: AP@10
0.9800: AP@50

  mAP:
0.7000: mAP@1
0.7600: mAP@5
0.7700: mAP@10
0.7040: mAP@50
---------------------

--- CLASSIFICATION --
  PER CLASS:
0.7855: dresses
0.3290: jackets and coats
0.6893: jeans
0.8989: pants
0.8200: shirts
0.0513: shorts
0.2191: skirts
0.7148: suits and blazers
0.7161: sweaters
0.5400: tops

Average top-1 accuracy: 0.5971


In [88]:
os.makedirs('/kaggle/working/utils', exist_ok=True)

In [104]:
!zip -r utils.zip /kaggle/working/utils

updating: kaggle/working/utils/ (stored 0%)
updating: kaggle/working/utils/char-CNN-RNN-embeddings-val.pickle (deflated 7%)
updating: kaggle/working/utils/char-CNN-RNN-embeddings.pickle (deflated 7%)
updating: kaggle/working/utils/weights.pickle (deflated 8%)
updating: kaggle/working/utils/model.pickle (deflated 8%)


In [102]:
from IPython.display import FileLink

In [103]:
FileLink('utils.zip')

/kaggle/working/checkpoints/utils.zip